In [1]:
kubectl apply -f $OLDPWD/simple-autoscaling.yaml

service.serving.knative.dev/autoscale-go created


In [3]:
# In Knative 0.2.x and prior versions, the `knative-ingressgateway` service was used instead of `istio-ingressgateway`.
INGRESSGATEWAY=knative-ingressgateway

# The use of `knative-ingressgateway` is deprecated in Knative v0.3.x.
# Use `istio-ingressgateway` instead, since `knative-ingressgateway`
# will be removed in Knative v0.4.
if kubectl get configmap config-istio -n knative-serving &> /dev/null; then
  INGRESSGATEWAY=istio-ingressgateway
fi

echo $INGRESSGATEWAY

istio-ingressgateway


In [4]:
export IP_ADDRESS=$(kubectl get node  --output 'jsonpath={.items[0].status.addresses[0].address}'):$(kubectl get svc $INGRESSGATEWAY --namespace istio-system   --output 'jsonpath={.spec.ports[?(@.port==80)].nodePort}')
echo $IP_ADDRESS

10.0.2.15:30363


In [6]:
kubectl get service.serving.knative.dev/autoscale-go
export HOST_URL=$(kubectl get route autoscale-go  --output jsonpath='{.status.domain}')
echo $HOST_URL

NAME           DOMAIN                             LATESTCREATED        LATESTREADY          READY   REASON
autoscale-go   autoscale-go.default.example.com   autoscale-go-qpzg6   autoscale-go-qpzg6   True    
autoscale-go.default.example.com


In [11]:
curl -H "Host: ${HOST_URL}" "http://${IP_ADDRESS}?sleep=100&prime=10000&bloat=5"
echo "curl -H \"Host: ${HOST_URL}\" http://${IP_ADDRESS}?sleep=100&prime=10000&bloat=5"

Slept for 100.38 milliseconds.
Allocated 5 Mb of memory.
The largest prime less than 10000 is 9973.
Slept for 100.09 milliseconds.
[3]+  Done                    curl -H "Host: ${HOST_URL}" http://${IP_ADDRESS}?sleep=100
curl -H "Host: autoscale-go.default.example.com" http://10.0.2.15:30363?sleep=100&prime=10000&bloat=5


In [14]:
hey -z 30s -c 50 \
  -host ${HOST_URL} \
  "http://${IP_ADDRESS}?sleep=100&prime=10000&bloat=5"


Summary:
  Total:	30.1173 secs
  Slowest:	0.4070 secs
  Fastest:	0.1024 secs
  Average:	0.1270 secs
  Requests/sec:	392.5317
  
  Total data:	1182157 bytes
  Size/request:	99 bytes

Response time histogram:
  0.102 [1]	|
  0.133 [9075]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.163 [2016]	|■■■■■■■■■
  0.194 [390]	|■■
  0.224 [150]	|■
  0.255 [87]	|
  0.285 [38]	|
  0.316 [16]	|
  0.346 [26]	|
  0.377 [3]	|
  0.407 [20]	|


Latency distribution:
  10% in 0.1082 secs
  25% in 0.1122 secs
  50% in 0.1194 secs
  75% in 0.1316 secs
  90% in 0.1507 secs
  95% in 0.1719 secs
  99% in 0.2499 secs

Details (average, fastest, slowest):
  DNS+dialup:	0.0001 secs, 0.1024 secs, 0.4070 secs
  DNS-lookup:	0.0000 secs, 0.0000 secs, 0.0000 secs
  req write:	0.0000 secs, 0.0000 secs, 0.0245 secs
  resp wait:	0.1267 secs, 0.1024 secs, 0.4069 secs
  resp read:	0.0001 secs, 0.0000 secs, 0.0164 secs

Status code distribution:
  [200]	11822 responses





In [15]:
kubectl apply -f $OLDPWD/cpu-autoscaling.yaml

service.serving.knative.dev/autoscale-go configured


In [17]:
hey -z 30s -c 50 \
  -host ${HOST_URL} \
  "http://${IP_ADDRESS}?sleep=100&prime=10000&bloat=5"


Summary:
  Total:	30.2166 secs
  Slowest:	0.6572 secs
  Fastest:	0.1131 secs
  Average:	0.2494 secs
  Requests/sec:	199.5591
  
  Total data:	603000 bytes
  Size/request:	100 bytes

Response time histogram:
  0.113 [1]	|
  0.168 [658]	|■■■■■■■■■■■■■■
  0.222 [1859]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.276 [1692]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.331 [989]	|■■■■■■■■■■■■■■■■■■■■■
  0.385 [496]	|■■■■■■■■■■■
  0.440 [208]	|■■■■
  0.494 [67]	|■
  0.548 [36]	|■
  0.603 [15]	|
  0.657 [9]	|


Latency distribution:
  10% in 0.1652 secs
  25% in 0.1953 secs
  50% in 0.2353 secs
  75% in 0.2902 secs
  90% in 0.3502 secs
  95% in 0.3927 secs
  99% in 0.4947 secs

Details (average, fastest, slowest):
  DNS+dialup:	0.0001 secs, 0.1131 secs, 0.6572 secs
  DNS-lookup:	0.0000 secs, 0.0000 secs, 0.0000 secs
  req write:	0.0001 secs, 0.0000 secs, 0.0490 secs
  resp wait:	0.2489 secs, 0.1112 secs, 0.6570 secs
  resp read:	0.0002 secs, 0.0000 secs, 0.0491 secs

Status code distribution:


In [22]:
kubectl apply -f $OLDPWD/scaling-bounds-autoscaling.yaml

service.serving.knative.dev/autoscale-go configured


In [23]:
hey -z 30s -c 100 \
  -host ${HOST_URL} \
  "http://${IP_ADDRESS}?sleep=100&prime=10000&bloat=5"


Summary:
  Total:	30.4160 secs
  Slowest:	1.7994 secs
  Fastest:	0.1218 secs
  Average:	0.4932 secs
  Requests/sec:	200.8808
  
  Total data:	611000 bytes
  Size/request:	100 bytes

Response time histogram:
  0.122 [1]	|
  0.290 [1249]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.457 [1881]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.625 [1435]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  0.793 [828]	|■■■■■■■■■■■■■■■■■■
  0.961 [411]	|■■■■■■■■■
  1.128 [202]	|■■■■
  1.296 [64]	|■
  1.464 [34]	|■
  1.632 [3]	|
  1.799 [2]	|


Latency distribution:
  10% in 0.2332 secs
  25% in 0.3128 secs
  50% in 0.4503 secs
  75% in 0.6291 secs
  90% in 0.8192 secs
  95% in 0.9610 secs
  99% in 1.2230 secs

Details (average, fastest, slowest):
  DNS+dialup:	0.0002 secs, 0.1218 secs, 1.7994 secs
  DNS-lookup:	0.0000 secs, 0.0000 secs, 0.0000 secs
  req write:	0.0001 secs, 0.0000 secs, 0.0522 secs
  resp wait:	0.4912 secs, 0.1216 secs, 1.7993 secs
  resp read:	0.0001 secs, 0.0000 secs, 0.0270 secs

Status code distributio